In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import *
from keras.models import load_model


import os
import time
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

#variables para guardar los resultados
tipo = pruebas[4]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Dataset de Amazon

In [3]:
print tipo
print dataset_name
print data_path
print dims

sda
amazon
data
3000


In [4]:
# cargando dataset de Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)
print X.shape
print "Todos los datos disponibles obtenidos"

(27677, 3000)
Todos los datos disponibles obtenidos


In [6]:
# se genera un sub-conjunto del dataset
spd = 10000 / len(domains)

X_small = None
for domain in domains:
    temp = dataset_object.get_all_domain_X(domain)
    indexes = np.random.choice(temp.shape[0], spd)
    
    if X_small is None:
        X_small = temp[indexes,:]
    else:
        X_small = np.vstack((X_small, temp[indexes,:]))

X_small = np.asarray(X_small)
print X_small.shape

(10000, 3000)


In [7]:
# se divide el dataset para los datos de entrenamiento y validacion del SDA
X_train, X_val, _, _ = train_test_split(X_small, np.zeros(X_small.shape[0]), test_size=0.2, random_state=42)

print X_train.shape
print X_val.shape

(8000, 3000)
(2000, 3000)


In [8]:
sda_paths = os.path.join(models_path, tipo, "%s_paths.pkl" % dataset_name)

if os.path.exists(sda_paths):
    print "Cargando rutas de modelos adaptados."
    saved_paths = joblib.load(sda_paths)
else:
    print "Creando modelos de adaptacion..."

    noises = [0.3 , 0.5, 0.8]
    layers = [[int(dims/2)], [int(dims/2), int(dims/4)]]
    epochs = [50, 25]

    parametros = {
        'noise': noises,
        'layers': layers,
        'epochs': epochs,
    }
    
    saved_paths = sda_pseudo_grid_search(X_train, X_val, parametros, models_path, tipo, dataset_name)

    print "\tModelos creados.\n"
    
    joblib.dump(saved_paths, sda_paths)
    print "\tGuardando rutas en %s" % sda_paths

print "\nCreacion de modelos terminada."

Creando modelos de adaptacion...
pr: 0.300 - epochs: 50 - layers: [1500]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/amazon_ae_0.h5
	Guardando encoder en models/sda/amazon_e_0.h5
pr: 0.300 - epochs: 25 - layers: [1500]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/amazon_ae_1.h5
	Guardando encoder en models/sda/amazon_e_1.h5
pr: 0.300 - epochs: 50 - layers: [1500, 750]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/amazon_ae_2.h5
	Guardando encoder en models/sda/amazon_e_2.h5
pr: 0.300 - epochs: 25 - layers: [1500, 750]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/amazon_ae_3.h5
	Guardando encoder en models/sda/amazon_e_3.h5
pr: 0.500 - epochs: 50 - layers: [1500]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/amazon_ae_4.h5
	Guardando encoder en models/sda/amazon_e_4.h5
pr: 0.500 - epochs: 25 - layers: [1500]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/amazon_ae_5.h5
	Guardan

In [9]:
saved_paths

[{'autoencoder': 'models/sda/amazon_ae_0.h5',
  'encoder': 'models/sda/amazon_e_0.h5'},
 {'autoencoder': 'models/sda/amazon_ae_1.h5',
  'encoder': 'models/sda/amazon_e_1.h5'},
 {'autoencoder': 'models/sda/amazon_ae_2.h5',
  'encoder': 'models/sda/amazon_e_2.h5'},
 {'autoencoder': 'models/sda/amazon_ae_3.h5',
  'encoder': 'models/sda/amazon_e_3.h5'},
 {'autoencoder': 'models/sda/amazon_ae_4.h5',
  'encoder': 'models/sda/amazon_e_4.h5'},
 {'autoencoder': 'models/sda/amazon_ae_5.h5',
  'encoder': 'models/sda/amazon_e_5.h5'},
 {'autoencoder': 'models/sda/amazon_ae_6.h5',
  'encoder': 'models/sda/amazon_e_6.h5'},
 {'autoencoder': 'models/sda/amazon_ae_7.h5',
  'encoder': 'models/sda/amazon_e_7.h5'},
 {'autoencoder': 'models/sda/amazon_ae_8.h5',
  'encoder': 'models/sda/amazon_e_8.h5'},
 {'autoencoder': 'models/sda/amazon_ae_9.h5',
  'encoder': 'models/sda/amazon_e_9.h5'},
 {'autoencoder': 'models/sda/amazon_ae_10.h5',
  'encoder': 'models/sda/amazon_e_10.h5'},
 {'autoencoder': 'models/sda/a

In [10]:
best_sda_paths = os.path.join(models_path, tipo, "best_%s_paths.pkl" % dataset_name )
if os.path.exists(best_sda_paths):
    #se cargan las rutas de los modelos guardados
    domain_sda_paths = joblib.load(best_sda_paths)   
else:
    domain_sda_paths = []
    #por cada dominio
    for domain in domains:
        #por cada modelo
        print "Obteniendo mejor clasificador para %s..." % domain
        
        i = 1
        best_score = 0
        best_encoder = None
        
        for temp_path in saved_paths:
            #autoencoder = load_model(temp_path['autoencoder'])
            encoder = load_model(temp_path['encoder'])

        
            X_tr = np.asarray(labeled[domain]['X_tr'].todense())
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[domain]['X_ts'].todense())
            
            #se adapta el dominio segun el modelo
            X_deep = encoder.predict(X_tr)

            #se crea un clasificador y se obtiene su score
            new_clf = get_best_score(X_deep, y_tr, clasifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            
            print "%d - score: %.4f" % (i, new_score)
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_encoder = encoder
                
            i= i+1
            
        best_encoder_path = os.path.join(models_path, tipo, "%s_%s_encoder.pkl" % (dataset_name, domain))
        print "Guardando en %s" % best_encoder_path
        encoder.save(best_encoder_path)
        
        domain_sda_paths.append(best_encoder_path)
        
    print "Guardando rutas en %s" % best_sda_paths
    joblib.dump(domain_sda_paths, best_sda_paths)
    
print "\nOperacion terminada.\n"

Obteniendo mejor clasificador para electronics...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/keras/models.py:240: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


1 - score: 0.8331
2 - score: 0.8100
3 - score: 0.7812
4 - score: 0.7831
5 - score: 0.8075
6 - score: 0.8000
7 - score: 0.7719
8 - score: 0.7850
9 - score: 0.8131
10 - score: 0.8275
11 - score: 0.7650
12 - score: 0.7712
Guardando en models/sda/amazon_electronics_encoder.pkl
Obteniendo mejor clasificador para dvd...
1 - score: 0.7562
2 - score: 0.7450
3 - score: 0.7056
4 - score: 0.7388
5 - score: 0.7475
6 - score: 0.7438
7 - score: 0.7125
8 - score: 0.7131
9 - score: 0.7512
10 - score: 0.7394
11 - score: 0.7306
12 - score: 0.7137
Guardando en models/sda/amazon_dvd_encoder.pkl
Obteniendo mejor clasificador para kitchen...
1 - score: 0.8250
2 - score: 0.8475
3 - score: 0.7869
4 - score: 0.7950
5 - score: 0.8350
6 - score: 0.8350
7 - score: 0.8025
8 - score: 0.7856
9 - score: 0.8375
10 - score: 0.8225
11 - score: 0.7950
12 - score: 0.7919
Guardando en models/sda/amazon_kitchen_encoder.pkl
Obteniendo mejor clasificador para books...
1 - score: 0.7406
2 - score: 0.7444
3 - score: 0.6806
4 - 

In [11]:
domain_sda_paths

['models/sda/amazon_electronics_encoder.pkl',
 'models/sda/amazon_dvd_encoder.pkl',
 'models/sda/amazon_kitchen_encoder.pkl',
 'models/sda/amazon_books_encoder.pkl']

In [12]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_sda_path = os.path.join(models_path, tipo, "%s_%s_encoder.pkl" % (dataset_name, domain))
    
    if os.path.exists(best_sda_path):
        print "Cargando mejor modelo para %s" % domain
        encoder = load_model(best_sda_path)
        
        print "Adaptando dominio"
        X_tr = np.asarray(labeled[domain]['X_tr'].todense())
        X_ts = np.asarray(labeled[domain]['X_ts'].todense())
        
        
        tr_reps = encoder.predict(X_tr)
        ts_reps = encoder.predict(X_ts)

        adapted[domain] = {
            'X_tr': tr_reps,
            'X_ts': ts_reps
        }
        
        i = i+1
    else:
        print "Generar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para electronics
Adaptando dominio
Cargando mejor modelo para dvd
Adaptando dominio
Cargando mejor modelo para kitchen
Adaptando dominio
Cargando mejor modelo para books
Adaptando dominio
4/4 dominios adaptados


In [13]:
df = pd.DataFrame(columns=dataframe_columns)

i = 0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 12" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapted[tgt]['X_ts']
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            print "Entrenando clasificador adaptado."
            svc_a = get_best_score(X_tr_a, y_tr, n_jobs=4)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['SDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            i+=1

print "\nPruebas completadas."

Tarea 1 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 2 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 3 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 4 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 5 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 6 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 7 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 8 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 9 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 10 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 11 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 12 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Pruebas completadas.


In [14]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,SDA,e->d,electronics,dvd,23.00,36.25,13.25
1,SDA,e->k,electronics,kitchen,17.50,26.00,8.50
2,SDA,e->b,electronics,books,21.25,37.50,16.25
3,SDA,d->e,dvd,electronics,20.00,30.00,10.00
4,SDA,d->k,dvd,kitchen,17.50,32.50,15.00
5,SDA,d->b,dvd,books,21.25,33.50,12.25
6,SDA,k->e,kitchen,electronics,20.00,26.00,6.00
7,SDA,k->d,kitchen,dvd,23.00,34.25,11.25
8,SDA,k->b,kitchen,books,21.25,35.25,14.00
9,SDA,b->e,books,electronics,20.00,42.50,22.50


In [15]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/sda/amazon.csv
Resultados guardados.
